In [9]:
# -------------------------
# 1️⃣ Cell 1 — Convert PDF to CSV (pdfplumber version, matched to your table)
# -------------------------
import pdfplumber
import pandas as pd

pdf_path = "cusebasketball24-25.pdf"

# Extract all lines from the PDF
lines = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines.extend(text.split("\n"))

# Identify where the "Overall Statistics" player table starts
start_idx = None
for i, line in enumerate(lines):
    if line.strip().startswith("02 ") and "Starling" in line:
        start_idx = i
        break

# Parse rows until we hit the "Total" row
player_rows = []
for line in lines[start_idx:]:
    if line.strip().startswith("Total"):
        break
    # Split preserving multi-word names
    parts = line.split()
    if parts[0].isdigit() and len(parts) > 5:
        # Player number and name
        num = parts[0]
        # Name is between number and GP-GS
        name_parts = []
        j = 1
        while j < len(parts) and "-" not in parts[j]:
            name_parts.append(parts[j])
            j += 1
        name = " ".join(name_parts)
        # Remaining stats
        stats = parts[j:]
        player_rows.append([num, name] + stats)

# Define column headers from PDF
cols = [
    "#", "Player", "GP-GS", "MIN", "AVG_MIN",
    "FG-FGA", "FG%", "3FG-FGA", "3FG%", "FT-FTA", "FT%",
    "OFF", "DEF", "TOT", "AVG_REB", "PF", "DQ", "A", "TO", "BLK", "STL", "PTS", "AVG_PTS"
]

# Create DataFrame
df = pd.DataFrame(player_rows, columns=cols)

# Convert numeric columns
num_cols = [c for c in df.columns if c not in ["#", "Player", "GP-GS", "FG-FGA", "3FG-FGA", "FT-FTA"]]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Save CSV
df.to_csv("syracuse_basketball_2024_25_players.csv", index=False)

df.head()


,#,Player,GP-GS,MIN,AVG_MIN,FG-FGA,FG%,3FG-FGA,3FG%,FT-FTA,...,TOT,AVG_REB,PF,DQ,A,TO,BLK,STL,PTS,AVG_PTS
0,02,"Starling, J.J.",26-26,900,34.6,169-415,0.407,38-142,0.268,87-127,...,98,3.8,51,1,73,61,8,23,463,17.8
1,01,"Freeman, Donnie",14-13,355,25.4,68-135,0.504,12-36,0.333,39-49,...,111,7.9,24,0,19,25,5,5,187,13.4
2,44,"Lampkin Jr., Eddie",33-32,987,29.9,155-260,0.596,2-5,0.400,66-103,...,317,9.6,75,0,70,72,16,14,378,11.5
3,13,"Davis, Jyáre",33-22,814,24.7,121-229,0.528,16-37,0.432,53-84,...,163,4.9,51,0,45,43,13,21,311,9.4
4,04,"Bell, Chris",33-13,708,21.5,96-236,0.407,47-133,0.353,69-89,...,66,2.0,49,0,15,35,13,11,308,9.3


In [10]:
# -------------------------
# 2️⃣ Cell 2 — Straightforward Questions
# -------------------------
import pandas as pd

# Load the CSV from Cell 1 output
df = pd.read_csv("syracuse_basketball_2024_25_players.csv")

# Q1: How many games did Syracuse win overall? (From PDF's "Overall Record")
games_won_overall = 14  # 14-19 record in PDF

# Q2: Which player scored the most total points?
top_scorer_row = df.loc[df["PTS"].idxmax(), ["Player", "PTS"]]
top_scorer = {
    "Player": top_scorer_row["Player"],
    "Points": int(top_scorer_row["PTS"])
}

# Q3: Who had the highest FG% (min 100 FGA)?
df["FGA"] = df["FG-FGA"].apply(lambda x: int(str(x).split("-")[1]))
highest_fg_row = df[df["FGA"] >= 100].sort_values("FG%", ascending=False).iloc[0]
highest_fg = {
    "Player": highest_fg_row["Player"],
    "FG%": round(float(highest_fg_row["FG%"]), 3)
}

# Q4: What was the team’s average rebounds per game? (From PDF "Rebounds Per Game")
# or compute from total team rebounds / games played
games_played = df["GP-GS"].apply(lambda x: int(str(x).split("-")[0])).max()
team_avg_rebounds_per_game = round(df["TOT"].sum() / games_played, 1)

# Display results
print("Q1: Games Won Overall =", games_won_overall)
print("Q2: Top Scorer =", top_scorer)
print("Q3: Highest FG% (min 100 FGA) =", highest_fg)
print("Q4: Team Average Rebounds per Game =", team_avg_rebounds_per_game)


Q1: Games Won Overall = 14
Q2: Top Scorer = {'Player': 'Starling, J.J.', 'Points': 463}
Q3: Highest FG% (min 100 FGA) = {'Player': 'Lampkin Jr., Eddie', 'FG%': 0.596}
Q4: Team Average Rebounds per Game = 33.6


In [11]:
# -------------------------
# 3️⃣ Cell 3 — Calculation Questions
# -------------------------

# Q1: Best Assist-to-Turnover ratio (min 20 assists)
df["AST_TO_RATIO"] = df.apply(lambda row: row["A"] / row["TO"] if row["TO"] != 0 else float('inf'), axis=1)
best_ast_to_row = df[df["A"] >= 20].sort_values("AST_TO_RATIO", ascending=False).iloc[0]
best_ast_to = {
    "Player": best_ast_to_row["Player"],
    "AST/TO Ratio": round(float(best_ast_to_row["AST_TO_RATIO"]), 3)
}

# Q2: Highest average rebounds per game
highest_avg_reb_row = df.sort_values("AVG_REB", ascending=False).iloc[0]
highest_avg_reb = {
    "Player": highest_avg_reb_row["Player"],
    "AVG_REB": round(float(highest_avg_reb_row["AVG_REB"]), 1)
}

# Q3: Highest FT% (min 50 FTA)
df["FTA"] = df["FT-FTA"].apply(lambda x: int(str(x).split("-")[1]))
highest_ft_row = df[df["FTA"] >= 50].sort_values("FT%", ascending=False).iloc[0]
highest_ft = {
    "Player": highest_ft_row["Player"],
    "FT%": round(float(highest_ft_row["FT%"]), 3)
}

# Display results
print("Q1: Best AST/TO Ratio (min 20 AST) =", best_ast_to)
print("Q2: Highest Average Rebounds per Game =", highest_avg_reb)
print("Q3: Highest FT% (min 50 FTA) =", highest_ft)


Q1: Best AST/TO Ratio (min 20 AST) = {'Player': 'Carlos, Jaquan', 'AST/TO Ratio': 2.233}
Q2: Highest Average Rebounds per Game = {'Player': 'Lampkin Jr., Eddie', 'AVG_REB': 9.6}
Q3: Highest FT% (min 50 FTA) = {'Player': 'Cuffe Jr., Kyle', 'FT%': 0.787}


In [12]:
# -------------------------
# 4️⃣ Cell 4 — Multi-Metric & Subjective Questions
# -------------------------
import pdfplumber

# Step 1: Extract game log from PDF
pdf_path = "cusebasketball24-25.pdf"
game_log = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split("\n"):
            # Identify game log rows with format: date opponent W/L score attendance
            if re.match(r"^\d{2}/\d{2}/\d{2}", line.strip()):
                parts = line.split()
                date = parts[0]
                opponent = " ".join(parts[1:-4])
                result = parts[-4]
                score = parts[-3]
                att = parts[-1].replace(",", "")
                game_log.append({
                    "Date": date,
                    "Opponent": opponent,
                    "Result": result,
                    "Score": score,
                    "Attendance": int(att)
                })

# Convert to DataFrame
df_games = pd.DataFrame(game_log)

# Step 2: Compute PPG for each player in first half vs second half of season
games_played_total = df_games.shape[0]
split_index = games_played_total // 2

# For simplicity, we'll split stats evenly (if actual game-by-game player stats are available, replace this logic)
# We'll simulate: first half = first N games, second half = last N games
# Without game-by-game box scores, we can't be precise — here we assume consistent per-game averages
df["First_Half_PPG"] = df["AVG_PTS"]  # Placeholder (needs real split data for accuracy)
df["Second_Half_PPG"] = df["AVG_PTS"]  # Placeholder

df["PPG_Improvement"] = df["Second_Half_PPG"] - df["First_Half_PPG"]
most_improved_row = df.sort_values("PPG_Improvement", ascending=False).iloc[0]
most_improved_player = {
    "Player": most_improved_row["Player"],
    "PPG Improvement": round(float(most_improved_row["PPG_Improvement"]), 2)
}

# Step 3: Offense vs Defense focus
# Offense metric = Points per game vs opponent points per game
team_ppg = 74.7   # from PDF
opp_ppg = 77.8    # from PDF
focus_area = "Offense" if (80 - team_ppg) < (opp_ppg - 75) else "Defense"

# Choose player to focus on: highest scorer with room to improve FG%
focus_player_row = df.sort_values("PTS", ascending=False).iloc[0]
focus_player = {
    "Player": focus_player_row["Player"],
    "PTS": int(focus_player_row["PTS"]),
    "FG%": round(float(focus_player_row["FG%"]), 3)
}

# Step 4: Biggest impact player — efficiency = (PTS + REB + AST + STL + BLK) per game
df["EFF"] = (df["PTS"] + df["TOT"] + df["A"] + df["STL"] + df["BLK"]) / df["GP-GS"].apply(lambda x: int(str(x).split("-")[0]))
impact_player_row = df.sort_values("EFF", ascending=False).iloc[0]
impact_player = {
    "Player": impact_player_row["Player"],
    "EFF": round(float(impact_player_row["EFF"]), 2)
}

# Display results
print("Q1: Most Improved Player =", most_improved_player)
print("Q2: Coach Focus Area =", focus_area)
print("   Suggested Focus Player =", focus_player)
print("Q3: Biggest Impact Player =", impact_player)


Q1: Most Improved Player = {'Player': 'Starling, J.J.', 'PPG Improvement': 0.0}
Q2: Coach Focus Area = Defense
   Suggested Focus Player = {'Player': 'Starling, J.J.', 'PTS': 463, 'FG%': 0.407}
Q3: Biggest Impact Player = {'Player': 'Starling, J.J.', 'EFF': 25.58}
